In [1]:
# TensorBoard
# グラフや訓練曲線を可視化する
# 可視化するにはグラフ定義、訓練の統計情報(MSEなど)をTensorBoardが読み込むログディレクトリに格納する
# 実行ごとに別のログディレクトリを使用する必要があるので、タイムスタンプを活用する

import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from datetime import datetime

# タイムスタンプの取得とログディレクトリ名の設定
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [2]:
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [4]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X") # 定数ではなくplaceholderとして定義 nは特徴量数8なのでshape(none,9)となる
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

In [5]:
y_pred = X @ theta
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()


In [6]:
# グラフ内にMSEを評価して　サマリ　と呼ばれるTensorBoard互換のログバイナリ列に書き込む
mse_summary = tf.summary.scalar('MSE', mse)
# ログディレクトリのログファイルにサマリを書き込むためのFileWriterを作成　相対パスと可視化したいグラフを引数に渡す
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [7]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [8]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [9]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch}) # 定期的にmse_summaryノードを評価する
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step) # 評価したサマリーノードをバイナリログファイル(イベントファイル)に書き込む
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval() 

In [10]:
file_writer.close()

In [11]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

In [32]:
# 別途pythonで作成した tensorboard.py を同じディレクト上に置いている
import tensorboard as tb

In [33]:
# 実行するとjupyter notebook 上でtensorBoardが見れる
# 学習曲線はterminalで見ることに注意！！

# terminalを起動してjupyterファイルのディレクトリに移動
# tensorboard --logdir tf_logs/
# 上記コマンド後に[http://localhost:6006] で見れる

tb.show_graph(tf.get_default_graph().as_graph_def())